In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/workspaces/SebastianBot")

In [2]:
from cloud.dependencies.clients import resolve_gmail_client


gmail = resolve_gmail_client()

In [3]:
from datetime import datetime
from sebastian.shared.gmail.query_builder import GmailQueryBuilder



query = (
    GmailQueryBuilder()
    .from_email("rueckgabe@amazon.de")
    .subject("Ihre Rücksendung von", exact=False)
    .after_date(datetime(2025, 12, 11))
    .build()
)
return_mails = gmail.fetch_mails(query)

In [4]:
return_mail = return_mails[0]
return_mail

FullMailResponse(id='19b12043c0b60f4a', threadId='19b12043c0b60f4a', labelIds=['CATEGORY_UPDATES', 'INBOX'], snippet='Amazon Amazon Hallo Oliver, Ihr Rücksendeantrag wurde bestätigt.. Abgabe: Du kannst deine Rücksendung bis zum Jan 12 abgeben.. Ort: Beliebiger DHL Packstation. Die Rückerstattung erfolgt nach Abgabe', raw_payload={'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'oneironaut.oml@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:6a06:3b43:b0:75e:34f3:30e9 with SMTP id dx3csp73113pib;        Fri, 12 Dec 2025 02:03:46 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IGqz7LMA6r+0ogXhTWaizB0ieD4iNvjopmmvG28A8tJvmPIHOwUvM54kbikUaHWS6inf/t3'}, {'name': 'X-Received', 'value': 'by 2002:a05:6000:26ca:b0:429:d6dc:ae30 with SMTP id ffacd0b85a97d-42fb490f420mr1605511f8f.46.1765533826356;        Fri, 12 Dec 2025 02:03:46 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1765533826; cv=

In [5]:
return_mail.payload

'<!doctype html><html lang="de" dir="auto" xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office"><head><title></title><!--[if !mso]><!--><meta http-equiv="X-UA-Compatible" content="IE=edge"><!--<![endif]--><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1"><style type="text/css">#outlook a{padding:0}body{margin:0;padding:0;-webkit-text-size-adjust:100%;-ms-text-size-adjust:100%}table,td{border-collapse:collapse;mso-table-lspace:0pt;mso-table-rspace:0pt}img{border:0;height:auto;line-height:100%;outline:0;text-decoration:none;-ms-interpolation-mode:bicubic}p{display:block;margin:13px 0}</style><!--[if mso]>\r\n    <noscript>\r\n    <xml>\r\n    <o:OfficeDocumentSettings>\r\n      <o:AllowPNG/>\r\n      <o:PixelsPerInch>96</o:PixelsPerInch>\r\n    </o:OfficeDocumentSettings>\r\n    </xml>\r\n    </noscript>\r\n    <![endif]--><!--[if l

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(return_mail.payload, "html.parser")
text = soup.get_text()

In [7]:
text

'              Hallo Oliver, \r\n Ihr Rücksendeantrag wurde bestätigt.. \r\n Abgabe: Du kannst deine Rücksendung bis zum Jan 12 abgeben.. \r\n Ort: Beliebiger DHL Packstation. \r\n Die Rückerstattung erfolgt nach Abgabe deiner Rücksendung. \xa0Abbrechen, um Rücksendung zu ändern       Rückgabecode\xa0 \xa0     Zusammenfassung deiner Rückgabe \xa0 Bestellnummer 304-2524474-7155561 \xa0  Zwischensumme der Erstattung39,99 €Versand:0,00 €Geschätzte Gesamtsumme der Rückerstattung*39,99 € \xa0 \xa0  \xa0 KOSTENLOSE Rücksendung: Sie können den Artikel kostenlos zurücksenden \xa0 Der Betrag in Höhe von 39,99 € wird Ihrem Amazon-Konto als Guthaben gutgeschrieben. \xa0  \xa0 ^Der Erstattungsbetrag kann je nach Versand- oder Rücksendekosten variieren. Weitere Informationen. \xa0 Rückgabe bis: \xa0 12. Januar 2026 \xa0 Abgabestandort \xa0 DHL Abgabe an Packstation – weder Verpackung noch Drucker erforderlich Artikel in deinem Rücksendeantrag \xa0   \xa0Levoit Top-Fill Luftbefeuchter, 23dB... \xa0A

In [8]:
from cloud.dependencies.clients import resolve_gemini_client


gemini = resolve_gemini_client()

In [9]:
from sebastian.usecases.ReturnTracker.models import ReturnData

In [10]:
prompt = f"""Given the following email text, extract all relevant information:
--- email text start ---
{text}
--- email text end ---
"""

In [11]:
r = gemini.get_response(prompt, response_schema=ReturnData)

In [12]:
r.item

ReturnData(return_date='12. Januar 2026', order_number='304-2524474-7155561', pickup_location='DHL Abgabe an Packstation – weder Verpackung noch Drucker erforderlich', item_title='Levoit Top-Fill Luftbefeuchter, 23dB...')

## Service Usage

In [13]:
from cloud.dependencies.services import resolve_return_tracker_service

service = resolve_return_tracker_service()

In [14]:
from datetime import timedelta

result = service.get_recent_returns(time_back=timedelta(days=30))
result

Result(item=[ReturnData(return_date='12. Januar 2026', order_number='304-2524474-7155561', pickup_location='DHL Abgabe an Packstation – weder Verpackung noch Drucker erforderlich', item_title='Levoit Top-Fill Luftbefeuchter, 23dB...')], errors=[])

In [15]:
result = service.get_recent_returns(time_back=timedelta(days=360))

In [ ]:
assert result.item is not None
assert len(result.errors) == 0

len(result.item)

14